In [70]:
import cv2
import numpy as np
from scipy import ndimage

def removeDuplicates(listofElements):
    uniqueList = []
    for elem in listofElements:
        if elem not in uniqueList:
            uniqueList.append(elem)       
    return uniqueList



def findFomular(img):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ##  threshold
    th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

    ##  minAreaRect on the nozeros
    pts = cv2.findNonZero(threshed)
    ret = cv2.minAreaRect(pts)

    (cx,cy), (w,h), ang = ret
    if ang < -50:
        ang = 0

    ##  Find rotated matrix, do rotation
    M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
    rotated = cv2.warpAffine(threshed, M, (img.shape[1], img.shape[0]))


    ##  find and draw the upper and lower boundary of each lines
    hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)

    th = 2
    H,W = img.shape[:2]
    uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
    lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

    rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)

    thres = 0
    for i in range(1,len(uppers)):
        try:
            thres += (uppers[i+1] - lowers[i])
        except:
            continue

    if len(uppers) > 0:
        thres = thres/len(uppers)

    y_hs = []
    for i in range(len(lowers)):
        y_h1 = []
        try:
            dis = abs(lowers[i] - uppers[i+1])
            if dis > thres:
                y_h1 = [uppers[i], lowers[i] - uppers[i]]
                y_h2 = [uppers[i+1], lowers[i+1] - uppers[i]]
                y_hs.append(y_h1)
                y_hs.append(y_h2)
        except:
            continue

    y_hss = []  

    if len(y_hs) > 0 :
        for i in range(len(y_hs) - 1):
            if y_hs[i][0] != y_hs[i+1][0]:
                y_hss.append(y_hs[i])
        y_hss.append(y_hs[len(y_hs)-1])

    #binary
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)

    #dilation
    kernel = np.ones((60,60), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)

    #find contours
    im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #sort contours
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    avr_h = 0
    a = 0
    for i, ctr in enumerate(sorted_ctrs):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr)
        avr_h += h
        a += 1

    avr_h = avr_h/a
    x_ws = []
    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr)  
        if h >= avr_h/3 and h <= avr_h*3:
            x_w = [x,w]
            x_ws.append(x_w)

    thres_x = 0 
    for bb in x_ws:
        thres_x += bb[1]
    thres_x = thres_x/len(x_ws)

    x_wss = []
    x_w = []
    for i in range(len(x_ws)):
        dif = 0
        for j in range(len(x_ws)):
            if i!=j and abs(x_ws[i][0] - x_ws[j][0]) < thres_x:
                if (x_ws[i][0] - x_ws[j][0]) >=  0 and (x_ws[i][1] - x_ws[j][1]) >=  0:       
                    x_w = [x_ws[j][0], x_ws[i][1]]
                if (x_ws[i][0] - x_ws[j][0]) >=  0 and (x_ws[i][1] - x_ws[j][1]) <=  0:       
                    x_w = [x_ws[j][0], x_ws[j][1]]
                if (x_ws[i][0] - x_ws[j][0]) <=  0 and (x_ws[i][1] - x_ws[j][1]) >=  0:       
                    x_w = [x_ws[i][0], x_ws[i][1]]
                if (x_ws[i][0] - x_ws[j][0]) <=  0 and (x_ws[i][1] - x_ws[j][1]) <=  0:       
                    x_w = [x_ws[i][0], x_ws[j][1]]
                x_wss.append(x_w)
                dif +=1
        if dif == 0:
            x_wss.append(x_ws[i])
    x_wss = removeDuplicates(x_wss)             
    print(y_hss)
    bbox = []
    bb = []
    if len(y_hss) > 0 and len(x_wss) > 0:
        for i in range(len(x_wss)):
            for j in range(len(y_hss)):
                bb = [x_wss[i][0], y_hss[j][0], x_wss[i][1], y_hss[j][1]]
                bbox.append(bb)
    elif len(y_hss) == 0:
        for i in range(len(x_wss)):
            bb = [x_wss[i][0], 0 , x_wss[i][1], img.shape[0]]
            bbox.append(bb)

    elif len(x_wss) == 0:
        for i in range(len(y_hss)):
            bb = [0, y_hss[j][0], img.shape[1], y_hss[j][1]]
            bbox.append(bb)
    bboxs = []
    for bb in bbox:
        if (bb[2] * bb[3]) >  (img.shape[0]*img.shape[1]/100):
            cv2.rectangle(img,(bb[0],bb[1]),( bb[0] + bb[2], bb[1] + bb[3] ),(90,0,255),2)
            bboxs.append(bb)

    cv2.imwrite("result.png", img)
    return bboxs

In [71]:
img = cv2.imread("/home/topica/Desktop/test_hpt4.jpg")
findFomular(img)

[[116, 20], [176, 62], [229, 2], [287, 52], [379, 1], [395, 18]]


[[100, 116, 371, 20],
 [100, 176, 371, 62],
 [100, 287, 371, 52],
 [100, 395, 371, 18],
 [521, 116, 392, 20],
 [521, 176, 392, 62],
 [521, 287, 392, 52],
 [521, 395, 392, 18]]

In [ ]:
import sys
import base64
import requests
import json
import ast
img_b64 =""
image_uri = "data:image/jpg;base64," + img_b64
r = requests.post("https://api.mathpix.com/v3/latex",
    data=json.dumps({'url': image_uri}),
    headers={app_id = 'jcarroll', "app_key" = '13f1584b2f9edb8220bf619c0b4e3d5a',
            "Content-type": "application/json"})
res = ast.literal_eval(json.dumps(json.loads(r.text), indent=4, sort_keys=True))